In [25]:
import pandas as pd
import numpy as np
from numpy.random import choice
import datetime
from datetime import datetime

In [26]:
#People away or are already presenting
away = ['Vicky', 'David', 'Dale', 'Bella', 'Pip', 'Michael'] # anyone on this list won't be selected
already_presenting = ['Tomas','Jakub'] # if you've already got a volunteer then put their name here so the right number will be chosen

In [27]:
nos = already_presenting + away
number_to_select = 3 - len(already_presenting) # max of three people per JC

In [28]:
#Read in excel file
xls = pd.ExcelFile('JC_Schedule.xlsx')

In [29]:
#Define dataframes
papers = pd.read_excel(xls, 'Schedule')
presenters = pd.read_excel(xls, 'Presenters')
groups = pd.read_excel(xls, 'Groups')

In [30]:
#Remove unnecessary information
presenters = presenters.drop(['Count Intake Contribution', 'Count weighted by Intake Size', 'Group'], axis=1)
presenters = presenters.ffill(axis=0)
presenters = presenters.drop_duplicates()
presenters = presenters.dropna()

In [31]:
#Remove postdocs - not compulsory 
presenters = presenters[~presenters['Intake Year'].str.contains("PostDoc", na=False)]
presenters = presenters.reset_index()

In [32]:
#Weight
presenters['Weight'] = 1/presenters['Count Papers']
presenters = presenters.replace(np.inf, 2.0) #put higher, non inf, weight to those who have not presented

In [33]:
#Get weights
if len(nos) > 0:
    pattern = '|'.join(nos)
else:
    pattern = ' '
print(pattern)
PT_ECS = '|'.join(['PT', 'ECS'])
presenters['Weight'][presenters['Name'].str.contains(pattern) == True] = 0.0 #People away or already presenting
presenters['Weight'][presenters['Notes'].str.contains(PT_ECS) == True] = 0.5*presenters['Weight'] #Weight part-time and ECS more fairly 

#ignore first years if before summer of their first year (June) - up to them to present
presenters['First summer'] = '01/06/'+(presenters['Intake Year']+1).astype(str)
format_str = '%d/%m/%Y'
b4_summer = []
for i in presenters['First summer']:
    date_obj = datetime.strptime(i, format_str)
    if datetime.today()<date_obj:
        b4_summer.append(True)
    else:
        b4_summer.append(False)
presenters['Before summer'] = b4_summer
presenters['Weight'][presenters['Before summer']==True] = 0.0 
presenters = presenters.drop(['First summer', 'Before summer'], axis=1)

Tomas|Jakub|Vicky|David|Dale|Bella|Pip|Michael


/home/clg1n18/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/clg1n18/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/clg1n18/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [34]:
presenters

,index,Intake Year,Name,Count Papers,Notes,Other,Weight
0,6,2015,Bella,13,Compulsory,No,0.000000
1,7,2015,Pete,10,Compulsory,No,0.100000
2,8,2016,James L,2,Compulsory,No,0.500000
3,9,2016,Marta,3,Compulsory,No,0.333333
4,10,2016,Michael,2,ECS,No,0.000000
5,11,2016,Miika,3,Compulsory,No,0.333333
6,12,2016,Noel,5,Compulsory,No,0.200000
7,13,2016,Pip,16,Compulsory,No,0.000000
8,14,2016,Vicky,0,ECS,Mat leave,0.000000
9,15,2017,Alessandra,4,Compulsory,No,0.250000


In [35]:
presenters_chosen = []
#loop over the number that need to be chosen so can do probability by weight and not get duplicates
for i in range(number_to_select):
    #probability
    presenters['Probability_{}'.format(i)] = presenters['Weight']/(presenters['Weight'].sum())
    #draw at random
    draw_1 = choice(presenters['Name'], 1, p=presenters['Probability_{}'.format(i)])
    #make list of presenters
    presenters_chosen.append(draw_1[0])
    #remove person selected
    presenters = presenters.drop(presenters[presenters['Name']==draw_1[0]].index)
    #print(presenters)

In [36]:
print('Already presenting: {}'.format(already_presenting))
print('First choice presenters: {}'.format(presenters_chosen))

Already presenting: ['Tomas', 'Jakub']
First choice presenters: ['Brian']
